# Revisão de código 

Olá! 

Meu nome é Suelen. Estou feliz em revisar seu projeto hoje!

Quando vejo um erro pela primeira vez, apenas aponto. Deixarei você encontrá-lo e corrigi-lo sozinho. Além disso, ao longo do texto, farei algumas observações sobre melhorias no código e também farei comentários sobre suas percepções sobre o assunto. Mas se você ainda não consegue lidar com essa tarefa, darei uma dica mais precisa na próxima iteração e também alguns exemplos práticos. Estarei aberto a feedbacks e discussões sobre o tema.

Você pode encontrar meus comentários em caixas verdes, amarelas ou vermelhas como estas:


<div class="alert alert-block alert-success">
<b>Comentário: </b> <a class="tocSkip"></a>

Sucesso. Tudo está correto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentário: </b> <a class="tocSkip"></a>

Observações. Algumas recomendações.
</div>

<div class="alert alert-block alert-danger">

<b>Comentário: </b> <a class="tocSkip"></a>

O bloco requer algumas correções. O trabalho não pode ser aceito com os comentários vermelhos.
</div>

Você pode me responder usando isto:

<div class="alert alert-block alert-info">
<b>Resposta do aluno</b> <a class="tocSkip"></a>
</div>



<div class="alert alert-block alert-success">
<b>Comentário geral: </b> <a class="tocSkip"></a>




**Parabéns pela conclusão do projeto, Charlys!**

Seu raciocínio ao planejar a solução foi bem estruturado, demonstrando uma boa compreensão do problema e das etapas necessárias para resolvê-lo. As escolhas dos modelos e das métricas foram adequadas ao contexto, e a implementação do código foi feita com clareza e precisão.

Excelente trabalho, continue evoluindo nessa jornada! 👏

</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


In [ ]:
personal = pd.read_csv('final_provider/personal.csv')
contract = pd.read_csv('final_provider/contract.csv')
phone = pd.read_csv('final_provider/phone.csv')
internet = pd.read_csv('final_provider/internet.csv')
df = personal.merge(contract, on='customerID') \
             .merge(phone, on='customerID', how='left') \
             .merge(internet, on='customerID', how='left')
print(df.info())
print(df.isnull().sum())
print(df.describe(include='all'))

In [ ]:
df['EndDate_flag'] = df['EndDate'].apply(lambda x: 0 if x == 'No' else 1)
sns.countplot(data=df, x='EndDate_flag')
plt.title("Distribuição da Rotatividade (Churn)")
plt.show()
df_numeric = df.select_dtypes(include=['float64', 'int64'])
sns.heatmap(df_numeric.corr(), annot=True, cmap='coolwarm')
plt.title("Mapa de Correlação")
plt.show()


In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)
df['MultipleLines'].fillna('No', inplace=True)

cat_cols = df.select_dtypes(include='object').columns.drop('customerID')
df[cat_cols] = df[cat_cols].fillna('No')
df_encoded = pd.get_dummies(df.drop(['customerID', 'BeginDate', 'EndDate'], axis=1), drop_first=True)

scaler = StandardScaler()
num_cols = ['MonthlyCharges', 'TotalCharges']
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

X = df_encoded.drop('EndDate_flag', axis=1)
y = df_encoded['EndDate_flag']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f'{name} - Accuracy média: {scores.mean():.4f}')


In [ ]:
best_model = GradientBoostingClassifier()
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de Confusão")
plt.show()

In [ ]:
X_cluster = df_encoded.drop('EndDate_flag', axis=1)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_cluster)

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_cluster)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis')
plt.title('Clusterização de Clientes (PCA)')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.show()

Previsão de Rotatividade de Clientes (Churn) - Interconnect

Este projeto tem como objetivo prever a **rotatividade (churn)** dos clientes da operadora fictícia **Interconnect**, com base em dados contratuais, pessoais e de uso dos serviços.

---

##  Descrição dos Dados

Os dados são compostos por quatro arquivos:

- **`personal.csv`**: informações pessoais dos clientes
- **`contract.csv`**: dados do contrato (tipo, duração, cobrança, churn)
- **`internet.csv`**: serviços de internet contratados
- **`phone.csv`**: serviços de telefonia contratados

---

##  1. Análise Exploratória dos Dados (EDA)

### Estrutura e Qualidade

- Verificamos os tipos de dados, formatos e existência de **valores ausentes**.
- Algumas colunas como `TotalCharges` estavam com tipo incorreto (string) e foram convertidas.
- Valores nulos foram tratados com substituição pela **mediana** ou marcados como "No".

###  Distribuições

- A variável alvo `churn` foi definida com base na coluna `EndDate` (`No` = cliente ativo, valor presente = churn).
- Visualizamos a distribuição dos clientes por gênero, tipo de contrato, serviços, e outras variáveis.

###  Outliers

- Foram verificados possíveis **outliers** em `MonthlyCharges` e `TotalCharges`, mas optamos por normalização ao invés de remoção.

### Correlações

- Usamos um **heatmap** para verificar correlações entre variáveis numéricas.
- `MonthlyCharges` e `TotalCharges` têm forte correlação, como esperado.

---

## 2. Pré-processamento de Dados

###  Limpeza e Tratamento

- `TotalCharges` transformada para numérico e preenchida.
- Dados categóricos preenchidos com "No" ou "Unknown".
- Dados normalizados com **StandardScaler**.

###  Codificação

- Variáveis categóricas foram convertidas com **one-hot encoding** (dummies).
- A variável alvo foi definida como `EndDate_flag` (0 = ativo, 1 = churn).

###  Divisão de Dados

- Usamos **train_test_split** com estratificação (70% treino, 30% teste).

---

## 3. Desenvolvimento e Treinamento de Modelos

Testamos 3 algoritmos de classificação:

- **Regressão Logística**
- **Random Forest**
- **Gradient Boosting**

Usamos **validação cruzada (5-fold)** para comparar acurácia entre os modelos.

---

##  4. Avaliação do Modelo

O **Gradient Boosting** obteve o melhor desempenho.

### Métricas no conjunto de teste:

- **Accuracy**
- **Precision**
- **Recall**
- **F1-Score**

Também geramos:

- **Relatório de classificação**
- **Matriz de confusão**

---

## 5. Desafio Adicional: Agrupamento de Clientes

Aplicamos um modelo de **K-Means Clustering** para segmentar os clientes em 3 grupos com base em comportamento e perfil.

### Etapas:

- Normalizamos os dados
- Reduzimos a dimensionalidade com **PCA**
- Visualizamos os clusters em 2D

---

## Conclusões

- É possível prever a rotatividade com boa precisão usando dados contratuais e de serviços.
- O modelo pode ser integrado a sistemas de marketing para **identificar clientes propensos ao churn** e **oferecer promoções personalizadas**.
- A clusterização pode auxiliar na **segmentação de mercado** para campanhas direcionadas.

---

## Próximos Passos

- Testar técnicas de **oversampling (SMOTE)** para melhorar recall de churn.
- Explorar modelos como **XGBoost** e **LightGBM**.
- Automatizar alertas com base nas previsões de churn.